# Compute Project Stats
*Ack: Derived from project_stats.ipynb and project_stats.py*

* This notebook uses the GitHub [GraphQL API](https://developer.github.com/v4/) to compute the number of open and 
  closed bugs pertaining to Kubeflow GitHub Projects
  * Stats are broken down by labels
* Results are plotted using [plotly](https://plot.ly)
  * Plots are currently published on plot.ly for sharing; they are publicly vieable by anyone
  
## Setup GitHub

* You will need a GitHub personal access token in order to use the GitHub API
* See these [instructions](https://help.github.com/articles/creating-a-personal-access-token-for-the-command-line/) for creating a personal access token
  * You will need the scopes:
    * repo
    * read:org    
* Set the environment variable `GITHUB_TOKEN` to pass your token to the code

## Setup Plot.ly Online

* In order to use plot.ly to publish the plot you need to create a plot.ly account and get an API key
* Follow plot.ly's [getting started guide](https://plot.ly/python/getting-started/)
* Store your API key in `~/.plotly/.credentials `

In [16]:
# Use plotly cufflinks to plot data frames
# https://plot.ly/ipython-notebooks/cufflinks/
# instructions for offline plotting
# https://plot.ly/python/getting-started/#initialization-for-offline-plotting
#
# Follow the instructions for online plotting:
# https://plot.ly/python/getting-started/
# You will need to setup an account
import plotly.graph_objs as go
import itertools
import math
import pandas

In [2]:
import project_stats

## Process issues from the specified Project

In [3]:
c = project_stats.ProjectStats(project="KF1.0")
c.main()

Make plots showing different groups of labels

* Columns are multi level indexes
* See [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html) for instructions on multilevel indexes
   * We specify a list of tuples where each tuple specifies the item to select at the corresponding level in the index

### Compute expected effort 


In [4]:
# Costs of effort labels in workdays per eng
effort_labels_costs = { 
    "effort/1-day" : 1,
    "effort/1-days" : 1,
    "effort/3-days" : 3,
    "effort/5-days" : 5,
    "effort/2-weeks" : 10,
# removed temporarily because no issues in KF1.0 refer to this label yet
#    "effort/2-weeks+" : 20
}

effort_labels = list(effort_labels_costs.keys())

In [15]:
def current_effort_distribution(stats_df, open_or_total='open'):
    return stats_df[open_or_total][effort_labels].tail(1)
    
def open_effort_distribution(stats_df):
    return current_effort_distribution(stats_df, 'open')

def total_effort_distribution(stats_df):
    return current_effort_distribution(stats_df, 'total')

def current_effort_weeks(stats_df, open_or_total='open'):
    e = current_effort_distribution(stats_df, open_or_total)
    days = sum([e.iloc[0, e.columns.get_loc(l)] * effort_labels_costs[l] for l in effort_labels])
    # consider 5 workdays in weeks
    return math.ceil(days / 5)

def open_effort_weeks(stats_df):
    return current_effort_weeks(stats_df, 'open')

def total_effort_weeks(stats_df):
    return current_effort_weeks(stats_df, 'total')

In [13]:
open_effort_distribution(c.stats)

label,effort/1-day,effort/1-days,effort/3-days,effort/5-days,effort/2-weeks
time,,,,,
2019-09-18 13:18:46,4,6,7,7,7


In [ ]:
total_effort_distribution(c.stats)

In [ ]:
print(open_effort_weeks(c.stats))

In [ ]:
print(total_effort_weeks(c.stats))

In [31]:
df = pandas.DataFrame({}, index = effort_labels)
df['open'] = open_effort_distribution(c.stats).values[0]
df['total'] = total_effort_distribution(c.stats).values[0]

data = [
   go.Bar(
    x=effort_labels,
    y=df[col].values
    ) for col in df.columns
    
]

go.Figure(data)

In [32]:
help(go.Bar)

Help on class Bar in module plotly.graph_objs:

class Bar(plotly.basedatatypes.BaseTraceType)
 |  Bar(arg=None, alignmentgroup=None, base=None, basesrc=None, cliponaxis=None, constraintext=None, customdata=None, customdatasrc=None, dx=None, dy=None, error_x=None, error_y=None, hoverinfo=None, hoverinfosrc=None, hoverlabel=None, hovertemplate=None, hovertemplatesrc=None, hovertext=None, hovertextsrc=None, ids=None, idssrc=None, insidetextanchor=None, insidetextfont=None, legendgroup=None, marker=None, meta=None, metasrc=None, name=None, offset=None, offsetgroup=None, offsetsrc=None, opacity=None, orientation=None, outsidetextfont=None, r=None, rsrc=None, selected=None, selectedpoints=None, showlegend=None, stream=None, t=None, text=None, textangle=None, textfont=None, textposition=None, textpositionsrc=None, textsrc=None, tsrc=None, uid=None, uirevision=None, unselected=None, visible=None, width=None, widthsrc=None, x=None, x0=None, xaxis=None, xcalendar=None, xsrc=None, y=None, y0=None